<a href="https://colab.research.google.com/github/jithinmjoison/fast_ai_from_foundations/blob/main/Varitional_Autoencoders_July_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (layers,  models, datasets, callbacks,losses,optimizers, metrics,)

from scipy.stats import norm

In [4]:
### Parameters
IMAGE_SIZE = 32
BATCH_SIZE = 100
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 5
BETA = 500

In [5]:
### Preparing the data
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
### Preprocess the data

def preprocess(imgs):
  """
  Normalise and reshape the images"

  """

  imgs = imgs.astype(np.float32) / 255.0
  imgs = np.pad(imgs, ((0,0),(2,2),(2,2)), constant_values =0.0)
  imgs = np.expand_dims( imgs, -1)
  return imgs



In [7]:
x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [8]:
### Building the sampling function to ensure smoothness

class Sampling(layers.Layer):
  def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [9]:
## Encoder###

# Encoder
encoder_input = layers.Input(
    shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name="encoder_input"
)
x = layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(
    encoder_input
)
x = layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]  # the decoder will need this!

x = layers.Flatten()(x)
z_mean = layers.Dense(EMBEDDING_DIM, name="z_mean")(x)
z_log_var = layers.Dense(EMBEDDING_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")
encoder.summary()



Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 32)           320       ['encoder_input[0][0]']       
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 64)             18496     ['conv2d[0][0]']              
                                                                                                  
 conv2d_2 (Conv2D)           (None, 4, 4, 128)            73856     ['conv2d_1[0][0]']            
                                                                                            

In [10]:
# Decoder
decoder_input = layers.Input(shape=(EMBEDDING_DIM,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3, 3), strides=2, activation="relu", padding="same"
)(x)
x = layers.Conv2DTranspose(
    64, (3, 3), strides=2, activation="relu", padding="same"
)(x)
x = layers.Conv2DTranspose(
    32, (3, 3), strides=2, activation="relu", padding="same"
)(x)
decoder_output = layers.Conv2D(
    1,
    (3, 3),
    strides=1,
    activation="sigmoid",
    padding="same",
    name="decoder_output",
)(x)

decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 2048)              6144      
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 32, 32, 32)        18464 

In [11]:
class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs):
        """Call the model on a particular input."""
        z_mean, z_log_var, z = encoder(inputs)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction

    def train_step(self, data):
        """Step run during training."""
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(
                BETA
                * losses.binary_crossentropy(
                    data, reconstruction, axis=(1, 2, 3)
                )
            )
            kl_loss = tf.reduce_mean(
                tf.reduce_sum(
                    -0.5
                    * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                    axis=1,
                )
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        """Step run during validation."""
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, reconstruction = self(data)
        reconstruction_loss = tf.reduce_mean(
            BETA
            * losses.binary_crossentropy(data, reconstruction, axis=(1, 2, 3))
        )
        kl_loss = tf.reduce_mean(
            tf.reduce_sum(
                -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                axis=1,
            )
        )
        total_loss = reconstruction_loss + kl_loss

        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [12]:
# Create a variational autoencoder
vae = VAE(encoder, decoder)


In [13]:
### Training the VAE
# Create a variational autoencoder
# Compile the variational autoencoder
optimizer = optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer)

In [14]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint",
    save_weights_only=False,
    save_freq="epoch",
    monitor="loss",
    mode="min",
    save_best_only=True,
    verbose=0,
)
tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

In [15]:
vae.fit(
    x_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(x_test, x_test),
    callbacks=[model_checkpoint_callback, tensorboard_callback],
)

Epoch 1/5
598/600 [============================>.] - ETA: 0s - total_loss: 163.2181 - reconstruction_loss: 158.8720 - kl_loss: 4.3461

600/600 [==============================] - 15s 12ms/step - total_loss: 163.1372 - reconstruction_loss: 158.7898 - kl_loss: 4.3474 - val_loss: 141.4922 - val_reconstruction_loss: 136.6592 - val_kl_loss: 4.8329
Epoch 2/5
600/600 [==============================] - ETA: 0s - total_loss: 135.8547 - reconstruction_loss: 130.9573 - kl_loss: 4.8974

600/600 [==============================] - 6s 9ms/step - total_loss: 135.8547 - reconstruction_loss: 130.9573 - kl_loss: 4.8974 - val_loss: 137.8294 - val_reconstruction_loss: 132.6640 - val_kl_loss: 5.1654
Epoch 3/5
596/600 [============================>.] - ETA: 0s - total_loss: 134.1713 - reconstruction_loss: 129.1396 - kl_loss: 5.0317

600/600 [==============================] - 6s 10ms/step - total_loss: 134.1733 - reconstruction_loss: 129.1422 - kl_loss: 5.0312 - val_loss: 136.3941 - val_reconstruction_loss: 131.3641 - val_kl_loss: 5.0300
Epoch 4/5
595/600 [============================>.] - ETA: 0s - total_loss: 133.3295 - reconstruction_loss: 128.2396 - kl_loss: 5.0899

600/600 [==============================] - 6s 9ms/step - total_loss: 133.3421 - reconstruction_loss: 128.2514 - kl_loss: 5.0907 - val_loss: 136.0899 - val_reconstruction_loss: 130.7117 - val_kl_loss: 5.3782
Epoch 5/5
597/600 [============================>.] - ETA: 0s - total_loss: 132.7922 - reconstruction_loss: 127.6374 - kl_loss: 5.1549

600/600 [==============================] - 6s 10ms/step - total_loss: 132.8128 - reconstruction_loss: 127.6575 - kl_loss: 5.1554 - val_loss: 135.8651 - val_reconstruction_loss: 130.4017 - val_kl_loss: 5.4635


In [16]:
# Save the final models
vae.save("./models/vae")
encoder.save("./models/encoder")
decoder.save("./models/decoder")

In [18]:
## Reconstruct using the varitional Autoencoder

# Select a subset of the test set
n_to_predict = 5000
example_images = x_test[:n_to_predict]
example_labels = y_test[:n_to_predict]

In [19]:
# Create autoencoder predictions and display
z_mean, z_log_var, reconstructions = vae.predict(example_images)
print("Example real clothing items")
display(example_images)
print("Reconstructions")
display(reconstructions)

157/157 [==============================] - 2s 6ms/step
Example real clothing items


array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.        ],
         [0.        ],
         [0.  

Reconstructions


array([[[[5.46269007e-07],
         [7.47199067e-08],
         [1.14492309e-06],
         ...,
         [1.89974344e-07],
         [1.37015959e-07],
         [8.95681442e-05]],

        [[3.03176040e-09],
         [3.43420792e-10],
         [2.67282747e-08],
         ...,
         [3.36837829e-08],
         [1.11536975e-08],
         [1.56417013e-06]],

        [[1.81063331e-09],
         [3.56337709e-10],
         [3.70376171e-07],
         ...,
         [1.48181452e-05],
         [4.60685715e-06],
         [6.77881544e-05]],

        ...,

        [[1.21737912e-07],
         [2.68072569e-08],
         [4.39663921e-07],
         ...,
         [1.40165852e-07],
         [1.46679469e-07],
         [7.79870425e-06]],

        [[3.08503502e-07],
         [2.58527191e-08],
         [1.55138196e-07],
         ...,
         [2.25216596e-07],
         [7.57640919e-08],
         [6.14354758e-06]],

        [[1.44901744e-04],
         [2.28571662e-05],
         [1.85182871e-05],
         ...,
 